In [1]:
#PP1 PCB Detection and Merging

#Import the libraries
import pandas as pd
import numpy as np
import scipy as sp

import os
import sys
import time

import csv
import datetime

import importlib

from matplotlib import pyplot as plt
from matplotlib import dates  as md 
from matplotlib.colors import LogNorm

In [2]:
#Import routines
import routine_rawaccl_to_binryseqPP1  
import routine_get_pcb_durPP1          
import routine_elim_spur_pcbPP1        
import routine_get_pcb_level_dataPP1 
import routine_rev_merge_pxmty_algoPP1 
import routine_merging_algoPP1         
# import routine_merge_pxmty_algoPP1

Start of definition of the rawaccl_to_binryseq procedure 2020-05-08 18:16:52.840957
Version currently used was compiled at 2019-04-23 08:26
Inputs are inp_vibr_df,downsample_rate,use_filt_accl_flag,use_quant_sig_flag,HYS_LOW_THRESH,HYS_HIGH_THRESH
End   of definition of the procedure 2020-05-08 18:16:52.841305
Start of defining procedure : 2020-05-08 18:16:52.867702
End   of defining procedure : 2020-05-08 18:16:52.867820
Start of defining procedure : 2020-05-08 18:16:52.878588
End   of defining procedure : 2020-05-08 18:16:52.878698
Start of defining procedure : 2020-05-08 18:16:52.902922
Version : 2019-04-23 12:50
End   of defining procedure : 2020-05-08 18:16:52.903087
Start of defining procedure : 2020-05-08 18:16:52.942506
Version : 2019-06-12 19:06
End   of defining procedure : 2020-05-08 18:16:52.942658
Start of defining procedure : 2020-05-08 18:16:52.980509
Using version 2019-04-06 08:30
End   of defining procedure : 2020-05-08 18:16:52.980969


In [3]:
pp1_vibr_file = pd.read_csv('PP1_IMU0506.csv',usecols = ['time','ax','ay','az'])
pp1_vibr_file = pp1_vibr_file.sort_values(by="time")
pp1_vibr_file = pp1_vibr_file.reset_index()
pp1_vibr_file = pp1_vibr_file.assign(net_accl = np.sqrt(pp1_vibr_file['ax']**2 + pp1_vibr_file['ay']**2 + pp1_vibr_file['az']**2))

In [4]:
pp1_vibr_file['time']=pd.to_datetime(pp1_vibr_file['time'])
pp1_vibr_file=pp1_vibr_file.rename(columns={'time':'timestamp'})

In [5]:
pp1_vibr_file

,index,timestamp,ax,ay,az,net_accl
0,0,2020-05-06 02:00:00.001553152,-0.001099,1.027832,0.034790,1.028421
1,1,2020-05-06 02:00:00.003091200,-0.001892,1.028320,0.036377,1.028965
2,2,2020-05-06 02:00:00.004076032,-0.001892,1.028320,0.036377,1.028965
3,3,2020-05-06 02:00:00.005014784,-0.001892,1.028320,0.036377,1.028965
4,4,2020-05-06 02:00:00.005961984,-0.001892,1.028320,0.036377,1.028965
5,5,2020-05-06 02:00:00.050237952,-0.003479,1.027710,0.034546,1.028296
6,6,2020-05-06 02:00:00.051776000,-0.003479,1.027710,0.034546,1.028296
7,7,2020-05-06 02:00:00.052768000,-0.002136,1.027832,0.034485,1.028413
8,8,2020-05-06 02:00:00.053710080,-0.002136,1.027832,0.034485,1.028413
9,9,2020-05-06 02:00:00.054670080,-0.002136,1.027832,0.034485,1.028413


In [6]:
filt_sig_pp1 = routine_rawaccl_to_binryseqPP1.rawaccl_to_binryseq(pp1_vibr_file,1,'N','Y',0.2,0.4)

START :  2020-05-08 18:17:45.660089
Mean      of normalized signal = 3.1090349850314747e-12
Deviation of normalized signal = 1.0000000000000289
-------------------------------------------------------
Performing Wiener filtering


/home/raghu/anaconda3/lib/python3.7/site-packages/scipy/signal/signaltools.py:491: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return x[reverse].conj()


Performing downsampling by : 1
Performing quantization
Mode = 0.05
0 : 0.1 | 2525725
0.1 : 0.2 | 765580
0.2 : 0.30000000000000004 | 77027
0.30000000000000004 : 0.4 | 0
0.4 : 0.5 | 36394
0.5 : 0.6 | 15443
0.6 : 0.7 | 6060
0.7 : 0.7999999999999999 | 4898
0.7999999999999999 : 0.8999999999999999 | 4812
0.8999999999999999 : 0.9999999999999999 | 5140
0.9999999999999999 : 1.0999999999999999 | 7188
1.0999999999999999 : 1.2 | 8141
1.2 : 1.3 | 10376
1.3 : 1.4000000000000001 | 26620
1.4000000000000001 : 1.5000000000000002 | 13998
1.5000000000000002 : 1.6000000000000003 | 16293
1.6000000000000003 : 1.7000000000000004 | 14624
1.7000000000000004 : 1.8000000000000005 | 14640
1.8000000000000005 : 1.9000000000000006 | 11486
1.9000000000000006 : 2.0000000000000004 | 9260
2.0000000000000004 : 2.1000000000000005 | 10312
2.1000000000000005 : 2.2000000000000006 | 8123
2.2000000000000006 : 2.3000000000000007 | 6966
2.3000000000000007 : 2.400000000000001 | 6196
2.400000000000001 : 2.500000000000001 | 3855
2.5

In [7]:
pcb_data_pp1 = routine_get_pcb_durPP1.get_pcb_dur(filt_sig_pp1.binry_sig)

End of processing at i = 94 j = 94


In [8]:
filt_sig_pp1['cor_binry_sig'] = routine_elim_spur_pcbPP1.elim_spur_pcb(filt_sig_pp1.binry_sig,pcb_data_pp1,950)

/home/raghu/Desktop/TCSstreaming/iiotstream2/streaming2/PP1/routine_elim_spur_pcbPP1.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  cor_binry_sig[STR_IND:END_IND] = 0.0


In [9]:
pcb_data_pp1 = routine_get_pcb_durPP1.get_pcb_dur(filt_sig_pp1.cor_binry_sig)

End of processing at i = 50 j = 50


In [10]:
pcb_level_pp1 = routine_get_pcb_level_dataPP1.get_pcb_level_data(filt_sig_pp1,pcb_data_pp1,50)

START :  2020-05-08 18:19:26.882884
str_index = 886407
local_str = 0
end_index = 3243282
local_end = 49
Mode = 975.0
local_mode = 975.0
------------------------
Mode calculated at : 2020-05-08 18:19:27.720170
Number of boards detected        = 50
Machine utilization factor       = 7.066594760559279 %
Average   board active duration  = 68.3472 secs
Median    board active duration  = 60.67333333333333 secs
Deviation board active duration  = 56.2514684662049 secs
END   :  2020-05-08 18:19:28.048782
Execution time = 0.019431633333333333 mins


In [11]:
pcb_level_pp1

,arvl_index,arvl_tmstmp,dptr_index,dptr_tmstmp,no_of_parts,proc_dur,maint_dur,weightage
0,886407,2020-05-06 04:24:14.822673152,892571,2020-05-06 04:25:14.999366144,1,60.176693,0.0,6.322051
1,905862,2020-05-06 04:27:24.721360896,907345,2020-05-06 04:27:39.137323008,1,14.415962,0.0,1.521026
2,908263,2020-05-06 04:27:48.098642176,914723,2020-05-06 04:28:51.072187904,1,62.973546,0.0,6.625641
3,1845476,2020-05-06 07:07:49.194881792,1846544,2020-05-06 07:07:59.625700096,1,10.430818,0.0,1.095385
4,2659210,2020-05-06 09:20:32.012749824,2664361,2020-05-06 09:21:22.269423872,1,50.256674,0.0,5.283077
5,2671795,2020-05-06 09:22:34.764625152,2672781,2020-05-06 09:22:44.412667136,1,9.648042,0.0,1.011282
6,2673130,2020-05-06 09:22:47.780901888,2674130,2020-05-06 09:22:57.628418048,1,9.847516,0.0,1.025641
7,2848141,2020-05-06 09:51:14.529961984,2854530,2020-05-06 09:52:16.831713792,1,62.301752,0.0,6.552821
8,2854675,2020-05-06 09:52:18.245349888,2860933,2020-05-06 09:53:19.278182144,1,61.032832,0.0,6.418462
9,2861106,2020-05-06 09:53:20.939009792,2867330,2020-05-06 09:54:21.632956928,1,60.693947,0.0,6.383590


In [12]:
pcb_level_pp1.reset_index(inplace=True)
pcb_level_pp1

,index,arvl_index,arvl_tmstmp,dptr_index,dptr_tmstmp,no_of_parts,proc_dur,maint_dur,weightage
0,0,886407,2020-05-06 04:24:14.822673152,892571,2020-05-06 04:25:14.999366144,1,60.176693,0.0,6.322051
1,1,905862,2020-05-06 04:27:24.721360896,907345,2020-05-06 04:27:39.137323008,1,14.415962,0.0,1.521026
2,2,908263,2020-05-06 04:27:48.098642176,914723,2020-05-06 04:28:51.072187904,1,62.973546,0.0,6.625641
3,3,1845476,2020-05-06 07:07:49.194881792,1846544,2020-05-06 07:07:59.625700096,1,10.430818,0.0,1.095385
4,4,2659210,2020-05-06 09:20:32.012749824,2664361,2020-05-06 09:21:22.269423872,1,50.256674,0.0,5.283077
5,5,2671795,2020-05-06 09:22:34.764625152,2672781,2020-05-06 09:22:44.412667136,1,9.648042,0.0,1.011282
6,6,2673130,2020-05-06 09:22:47.780901888,2674130,2020-05-06 09:22:57.628418048,1,9.847516,0.0,1.025641
7,7,2848141,2020-05-06 09:51:14.529961984,2854530,2020-05-06 09:52:16.831713792,1,62.301752,0.0,6.552821
8,8,2854675,2020-05-06 09:52:18.245349888,2860933,2020-05-06 09:53:19.278182144,1,61.032832,0.0,6.418462
9,9,2861106,2020-05-06 09:53:20.939009792,2867330,2020-05-06 09:54:21.632956928,1,60.693947,0.0,6.383590


In [13]:
pcb_level_pp1.drop('index', axis=1, inplace=True)

In [14]:
pcb_level_pp1

,arvl_index,arvl_tmstmp,dptr_index,dptr_tmstmp,no_of_parts,proc_dur,maint_dur,weightage
0,886407,2020-05-06 04:24:14.822673152,892571,2020-05-06 04:25:14.999366144,1,60.176693,0.0,6.322051
1,905862,2020-05-06 04:27:24.721360896,907345,2020-05-06 04:27:39.137323008,1,14.415962,0.0,1.521026
2,908263,2020-05-06 04:27:48.098642176,914723,2020-05-06 04:28:51.072187904,1,62.973546,0.0,6.625641
3,1845476,2020-05-06 07:07:49.194881792,1846544,2020-05-06 07:07:59.625700096,1,10.430818,0.0,1.095385
4,2659210,2020-05-06 09:20:32.012749824,2664361,2020-05-06 09:21:22.269423872,1,50.256674,0.0,5.283077
5,2671795,2020-05-06 09:22:34.764625152,2672781,2020-05-06 09:22:44.412667136,1,9.648042,0.0,1.011282
6,2673130,2020-05-06 09:22:47.780901888,2674130,2020-05-06 09:22:57.628418048,1,9.847516,0.0,1.025641
7,2848141,2020-05-06 09:51:14.529961984,2854530,2020-05-06 09:52:16.831713792,1,62.301752,0.0,6.552821
8,2854675,2020-05-06 09:52:18.245349888,2860933,2020-05-06 09:53:19.278182144,1,61.032832,0.0,6.418462
9,2861106,2020-05-06 09:53:20.939009792,2867330,2020-05-06 09:54:21.632956928,1,60.693947,0.0,6.383590


In [15]:
tempdf=pcb_level_pp1
tempdf['nmodes']=0
tempdf

,arvl_index,arvl_tmstmp,dptr_index,dptr_tmstmp,no_of_parts,proc_dur,maint_dur,weightage,nmodes
0,886407,2020-05-06 04:24:14.822673152,892571,2020-05-06 04:25:14.999366144,1,60.176693,0.0,6.322051,0
1,905862,2020-05-06 04:27:24.721360896,907345,2020-05-06 04:27:39.137323008,1,14.415962,0.0,1.521026,0
2,908263,2020-05-06 04:27:48.098642176,914723,2020-05-06 04:28:51.072187904,1,62.973546,0.0,6.625641,0
3,1845476,2020-05-06 07:07:49.194881792,1846544,2020-05-06 07:07:59.625700096,1,10.430818,0.0,1.095385,0
4,2659210,2020-05-06 09:20:32.012749824,2664361,2020-05-06 09:21:22.269423872,1,50.256674,0.0,5.283077,0
5,2671795,2020-05-06 09:22:34.764625152,2672781,2020-05-06 09:22:44.412667136,1,9.648042,0.0,1.011282,0
6,2673130,2020-05-06 09:22:47.780901888,2674130,2020-05-06 09:22:57.628418048,1,9.847516,0.0,1.025641,0
7,2848141,2020-05-06 09:51:14.529961984,2854530,2020-05-06 09:52:16.831713792,1,62.301752,0.0,6.552821,0
8,2854675,2020-05-06 09:52:18.245349888,2860933,2020-05-06 09:53:19.278182144,1,61.032832,0.0,6.418462,0
9,2861106,2020-05-06 09:53:20.939009792,2867330,2020-05-06 09:54:21.632956928,1,60.693947,0.0,6.383590,0


In [16]:
mod_actv_dur = routine_get_pcb_level_dataPP1.get_mode(pcb_data_pp1.pcb_actv_dur,50)/100

Mode = 975.0


In [17]:
mod_actv_dur

9.75

In [18]:
tseries = np.zeros(tempdf.shape[0])
tseries

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [19]:
for i, row in pcb_level_pp1.iterrows():
    nmodes=row.proc_dur/mod_actv_dur
    frac=nmodes%1
    nmodes=np.floor(nmodes)
    
    if (1-frac) <= .15:
        nmodes=np.floor(nmodes)+1
    
    if nmodes==1.0:
        continue
    tseries[i] = nmodes
tempdf['nmodes'] = tseries

In [20]:
tseries

array([ 6.,  0.,  6.,  0.,  5.,  0.,  0.,  6.,  6.,  6.,  4.,  0.,  4.,
        6.,  7.,  0.,  6.,  0.,  7.,  6.,  7., 14.,  7.,  3.,  5., 14.,
        7.,  4., 12.,  2.,  4.,  0.,  7., 14.,  0.,  0.,  2.,  2.,  0.,
        6., 21.,  2.,  2.,  5.,  2.,  0.,  4.,  4.,  0.,  0.])

In [21]:
temp_df1=tempdf[tempdf['nmodes']>1]

In [22]:
temp_df1

,arvl_index,arvl_tmstmp,dptr_index,dptr_tmstmp,no_of_parts,proc_dur,maint_dur,weightage,nmodes
0,886407,2020-05-06 04:24:14.822673152,892571,2020-05-06 04:25:14.999366144,1,60.176693,0.0,6.322051,6.0
2,908263,2020-05-06 04:27:48.098642176,914723,2020-05-06 04:28:51.072187904,1,62.973546,0.0,6.625641,6.0
4,2659210,2020-05-06 09:20:32.012749824,2664361,2020-05-06 09:21:22.269423872,1,50.256674,0.0,5.283077,5.0
7,2848141,2020-05-06 09:51:14.529961984,2854530,2020-05-06 09:52:16.831713792,1,62.301752,0.0,6.552821,6.0
8,2854675,2020-05-06 09:52:18.245349888,2860933,2020-05-06 09:53:19.278182144,1,61.032832,0.0,6.418462,6.0
9,2861106,2020-05-06 09:53:20.939009792,2867330,2020-05-06 09:54:21.632956928,1,60.693947,0.0,6.383590,6.0
10,2868319,2020-05-06 09:54:31.283346944,2872332,2020-05-06 09:55:10.432531200,1,39.149184,0.0,4.115897,4.0
12,2893103,2020-05-06 09:58:32.984549120,2897906,2020-05-06 09:59:19.788400896,1,46.803852,0.0,4.926154,4.0
13,2897936,2020-05-06 09:59:20.080846080,2904684,2020-05-06 10:00:25.891179008,1,65.810333,0.0,6.921026,6.0
14,2904806,2020-05-06 10:00:27.066859008,2912530,2020-05-06 10:01:42.429789952,1,75.362931,0.0,7.922051,7.0


In [23]:
split_times=[]
for i, row in temp_df1.iterrows():
    nmodes=int(row.nmodes)
    split_times.append(np.linspace(row.arvl_tmstmp.to_datetime64().astype('int'),row.dptr_tmstmp.to_datetime64().astype('int'),num=row.nmodes+1))

/home/raghu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: DeprecationWarning: object of type <class 'float'> cannot be safely interpreted as an integer.
  after removing the cwd from sys.path.


In [24]:
split_times

[array([1.58873905e+18, 1.58873906e+18, 1.58873907e+18, 1.58873908e+18,
        1.58873909e+18, 1.58873910e+18, 1.58873911e+18]),
 array([1.58873927e+18, 1.58873928e+18, 1.58873929e+18, 1.58873930e+18,
        1.58873931e+18, 1.58873932e+18, 1.58873933e+18]),
 array([1.58875683e+18, 1.58875684e+18, 1.58875685e+18, 1.58875686e+18,
        1.58875687e+18, 1.58875688e+18]),
 array([1.58875867e+18, 1.58875868e+18, 1.58875870e+18, 1.58875871e+18,
        1.58875872e+18, 1.58875873e+18, 1.58875874e+18]),
 array([1.58875874e+18, 1.58875875e+18, 1.58875876e+18, 1.58875877e+18,
        1.58875878e+18, 1.58875879e+18, 1.58875880e+18]),
 array([1.58875880e+18, 1.58875881e+18, 1.58875882e+18, 1.58875883e+18,
        1.58875884e+18, 1.58875885e+18, 1.58875886e+18]),
 array([1.58875887e+18, 1.58875888e+18, 1.58875889e+18, 1.58875890e+18,
        1.58875891e+18]),
 array([1.58875911e+18, 1.58875912e+18, 1.58875914e+18, 1.58875915e+18,
        1.58875916e+18]),
 array([1.58875916e+18, 1.58875917e+18, 

In [25]:
temp_df1.reset_index(inplace=True)

In [26]:
temp_df1

,index,arvl_index,arvl_tmstmp,dptr_index,dptr_tmstmp,no_of_parts,proc_dur,maint_dur,weightage,nmodes
0,0,886407,2020-05-06 04:24:14.822673152,892571,2020-05-06 04:25:14.999366144,1,60.176693,0.0,6.322051,6.0
1,2,908263,2020-05-06 04:27:48.098642176,914723,2020-05-06 04:28:51.072187904,1,62.973546,0.0,6.625641,6.0
2,4,2659210,2020-05-06 09:20:32.012749824,2664361,2020-05-06 09:21:22.269423872,1,50.256674,0.0,5.283077,5.0
3,7,2848141,2020-05-06 09:51:14.529961984,2854530,2020-05-06 09:52:16.831713792,1,62.301752,0.0,6.552821,6.0
4,8,2854675,2020-05-06 09:52:18.245349888,2860933,2020-05-06 09:53:19.278182144,1,61.032832,0.0,6.418462,6.0
5,9,2861106,2020-05-06 09:53:20.939009792,2867330,2020-05-06 09:54:21.632956928,1,60.693947,0.0,6.383590,6.0
6,10,2868319,2020-05-06 09:54:31.283346944,2872332,2020-05-06 09:55:10.432531200,1,39.149184,0.0,4.115897,4.0
7,12,2893103,2020-05-06 09:58:32.984549120,2897906,2020-05-06 09:59:19.788400896,1,46.803852,0.0,4.926154,4.0
8,13,2897936,2020-05-06 09:59:20.080846080,2904684,2020-05-06 10:00:25.891179008,1,65.810333,0.0,6.921026,6.0
9,14,2904806,2020-05-06 10:00:27.066859008,2912530,2020-05-06 10:01:42.429789952,1,75.362931,0.0,7.922051,7.0


In [27]:
PP_boardeventsSplit = pd.DataFrame()
for i, row in temp_df1.iterrows():
    nmodes=int(row.nmodes)
    wt=row.proc_dur/nmodes
#     en=row.energy/nmodes
#     if nmodes%2==0:
    for j in range(len(split_times[i])-1):
        ts1=pd.to_datetime(split_times[i][j])
        ts2=pd.to_datetime(split_times[i][j+1])
        PP_boardeventsSplit=PP_boardeventsSplit.append({"arvl_index":row.arvl_index, "arvl_tmstmp":ts1, "dptr_index":row.dptr_index, "dptr_tmstmp":ts2, "no_of_parts":1, "proc_dur": wt, "maint_dur":row.maint_dur, "weightage":wt/mod_actv_dur, "nmodes":1}, ignore_index=True)

In [28]:
PP_boardeventsSplit

,arvl_index,arvl_tmstmp,dptr_index,dptr_tmstmp,maint_dur,nmodes,no_of_parts,proc_dur,weightage
0,886407.0,2020-05-06 04:24:14.822673152,892571.0,2020-05-06 04:24:24.852122112,0.0,1.0,1.0,10.029449,1.028661
1,886407.0,2020-05-06 04:24:24.852122112,892571.0,2020-05-06 04:24:34.881570816,0.0,1.0,1.0,10.029449,1.028661
2,886407.0,2020-05-06 04:24:34.881570816,892571.0,2020-05-06 04:24:44.911019520,0.0,1.0,1.0,10.029449,1.028661
3,886407.0,2020-05-06 04:24:44.911019520,892571.0,2020-05-06 04:24:54.940468480,0.0,1.0,1.0,10.029449,1.028661
4,886407.0,2020-05-06 04:24:54.940468480,892571.0,2020-05-06 04:25:04.969917440,0.0,1.0,1.0,10.029449,1.028661
5,886407.0,2020-05-06 04:25:04.969917440,892571.0,2020-05-06 04:25:14.999366144,0.0,1.0,1.0,10.029449,1.028661
6,908263.0,2020-05-06 04:27:48.098642176,914723.0,2020-05-06 04:27:58.594233088,0.0,1.0,1.0,10.495591,1.076471
7,908263.0,2020-05-06 04:27:58.594233088,914723.0,2020-05-06 04:28:09.089824000,0.0,1.0,1.0,10.495591,1.076471
8,908263.0,2020-05-06 04:28:09.089824000,914723.0,2020-05-06 04:28:19.585415168,0.0,1.0,1.0,10.495591,1.076471
9,908263.0,2020-05-06 04:28:19.585415168,914723.0,2020-05-06 04:28:30.081006080,0.0,1.0,1.0,10.495591,1.076471


In [29]:
pcb_level_pp1.drop(tempdf[tempdf['nmodes']>1].index, inplace=True)

In [30]:
pcb_level_pp1

,arvl_index,arvl_tmstmp,dptr_index,dptr_tmstmp,no_of_parts,proc_dur,maint_dur,weightage,nmodes
1,905862,2020-05-06 04:27:24.721360896,907345,2020-05-06 04:27:39.137323008,1,14.415962,0.0,1.521026,0.0
3,1845476,2020-05-06 07:07:49.194881792,1846544,2020-05-06 07:07:59.625700096,1,10.430818,0.0,1.095385,0.0
5,2671795,2020-05-06 09:22:34.764625152,2672781,2020-05-06 09:22:44.412667136,1,9.648042,0.0,1.011282,0.0
6,2673130,2020-05-06 09:22:47.780901888,2674130,2020-05-06 09:22:57.628418048,1,9.847516,0.0,1.025641,0.0
11,2887813,2020-05-06 09:57:41.406988032,2889661,2020-05-06 09:57:59.399388160,1,17.992400,0.0,1.895385,0.0
15,2914251,2020-05-06 10:01:59.200624896,2915253,2020-05-06 10:02:08.995905024,1,9.795280,0.0,1.027692,0.0
17,2932338,2020-05-06 10:04:55.671915008,2934123,2020-05-06 10:05:13.075522048,1,17.403607,0.0,1.830769,0.0
31,3090026,2020-05-06 10:30:33.391310080,3091523,2020-05-06 10:30:48.012506112,1,14.621196,0.0,1.535385,0.0
34,3138996,2020-05-06 10:38:30.945561856,3140064,2020-05-06 10:38:41.376087040,1,10.430525,0.0,1.095385,0.0
35,3142589,2020-05-06 10:39:05.994461952,3143876,2020-05-06 10:39:18.525566208,1,12.531104,0.0,1.320000,0.0


In [31]:
pcb_level_pp1=pcb_level_pp1.append(PP_boardeventsSplit, ignore_index=True)

/home/raghu/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [32]:
pcb_level_pp1

,arvl_index,arvl_tmstmp,dptr_index,dptr_tmstmp,maint_dur,nmodes,no_of_parts,proc_dur,weightage
0,905862,2020-05-06 04:27:24.721360896,907345,2020-05-06 04:27:39.137323008,0.0,0.0,1,14.415962,1.521026
1,1845476,2020-05-06 07:07:49.194881792,1846544,2020-05-06 07:07:59.625700096,0.0,0.0,1,10.430818,1.095385
2,2671795,2020-05-06 09:22:34.764625152,2672781,2020-05-06 09:22:44.412667136,0.0,0.0,1,9.648042,1.011282
3,2673130,2020-05-06 09:22:47.780901888,2674130,2020-05-06 09:22:57.628418048,0.0,0.0,1,9.847516,1.025641
4,2887813,2020-05-06 09:57:41.406988032,2889661,2020-05-06 09:57:59.399388160,0.0,0.0,1,17.992400,1.895385
5,2914251,2020-05-06 10:01:59.200624896,2915253,2020-05-06 10:02:08.995905024,0.0,0.0,1,9.795280,1.027692
6,2932338,2020-05-06 10:04:55.671915008,2934123,2020-05-06 10:05:13.075522048,0.0,0.0,1,17.403607,1.830769
7,3090026,2020-05-06 10:30:33.391310080,3091523,2020-05-06 10:30:48.012506112,0.0,0.0,1,14.621196,1.535385
8,3138996,2020-05-06 10:38:30.945561856,3140064,2020-05-06 10:38:41.376087040,0.0,0.0,1,10.430525,1.095385
9,3142589,2020-05-06 10:39:05.994461952,3143876,2020-05-06 10:39:18.525566208,0.0,0.0,1,12.531104,1.320000


In [33]:
pcb_level_pp1 = pcb_level_pp1.sort_values(by="arvl_tmstmp")

In [34]:
pcb_level_pp1

,arvl_index,arvl_tmstmp,dptr_index,dptr_tmstmp,maint_dur,nmodes,no_of_parts,proc_dur,weightage
14,886407,2020-05-06 04:24:14.822673152,892571,2020-05-06 04:24:24.852122112,0.0,1.0,1,10.029449,1.028661
15,886407,2020-05-06 04:24:24.852122112,892571,2020-05-06 04:24:34.881570816,0.0,1.0,1,10.029449,1.028661
16,886407,2020-05-06 04:24:34.881570816,892571,2020-05-06 04:24:44.911019520,0.0,1.0,1,10.029449,1.028661
17,886407,2020-05-06 04:24:44.911019520,892571,2020-05-06 04:24:54.940468480,0.0,1.0,1,10.029449,1.028661
18,886407,2020-05-06 04:24:54.940468480,892571,2020-05-06 04:25:04.969917440,0.0,1.0,1,10.029449,1.028661
19,886407,2020-05-06 04:25:04.969917440,892571,2020-05-06 04:25:14.999366144,0.0,1.0,1,10.029449,1.028661
0,905862,2020-05-06 04:27:24.721360896,907345,2020-05-06 04:27:39.137323008,0.0,0.0,1,14.415962,1.521026
20,908263,2020-05-06 04:27:48.098642176,914723,2020-05-06 04:27:58.594233088,0.0,1.0,1,10.495591,1.076471
21,908263,2020-05-06 04:27:58.594233088,914723,2020-05-06 04:28:09.089824000,0.0,1.0,1,10.495591,1.076471
22,908263,2020-05-06 04:28:09.089824000,914723,2020-05-06 04:28:19.585415168,0.0,1.0,1,10.495591,1.076471


In [35]:
pcb_level_pp1.reset_index(inplace=True)

In [36]:
pcb_level_pp1.drop('index', axis=1, inplace=True)
pcb_level_pp1.drop('nmodes', axis=1, inplace=True)

In [37]:
pcb_level_pp1

,arvl_index,arvl_tmstmp,dptr_index,dptr_tmstmp,maint_dur,no_of_parts,proc_dur,weightage
0,886407,2020-05-06 04:24:14.822673152,892571,2020-05-06 04:24:24.852122112,0.0,1,10.029449,1.028661
1,886407,2020-05-06 04:24:24.852122112,892571,2020-05-06 04:24:34.881570816,0.0,1,10.029449,1.028661
2,886407,2020-05-06 04:24:34.881570816,892571,2020-05-06 04:24:44.911019520,0.0,1,10.029449,1.028661
3,886407,2020-05-06 04:24:44.911019520,892571,2020-05-06 04:24:54.940468480,0.0,1,10.029449,1.028661
4,886407,2020-05-06 04:24:54.940468480,892571,2020-05-06 04:25:04.969917440,0.0,1,10.029449,1.028661
5,886407,2020-05-06 04:25:04.969917440,892571,2020-05-06 04:25:14.999366144,0.0,1,10.029449,1.028661
6,905862,2020-05-06 04:27:24.721360896,907345,2020-05-06 04:27:39.137323008,0.0,1,14.415962,1.521026
7,908263,2020-05-06 04:27:48.098642176,914723,2020-05-06 04:27:58.594233088,0.0,1,10.495591,1.076471
8,908263,2020-05-06 04:27:58.594233088,914723,2020-05-06 04:28:09.089824000,0.0,1,10.495591,1.076471
9,908263,2020-05-06 04:28:09.089824000,914723,2020-05-06 04:28:19.585415168,0.0,1,10.495591,1.076471


In [38]:
pcb_merge_pp1 = pcb_level_pp1
mrg_binry_pp1 = filt_sig_pp1.cor_binry_sig

THRESHOLD = 0.95
HIGH_END  = 1.50

iter_one = 0
iter_two = 0

In [39]:
pcb_merge_pp1

,arvl_index,arvl_tmstmp,dptr_index,dptr_tmstmp,maint_dur,no_of_parts,proc_dur,weightage
0,886407,2020-05-06 04:24:14.822673152,892571,2020-05-06 04:24:24.852122112,0.0,1,10.029449,1.028661
1,886407,2020-05-06 04:24:24.852122112,892571,2020-05-06 04:24:34.881570816,0.0,1,10.029449,1.028661
2,886407,2020-05-06 04:24:34.881570816,892571,2020-05-06 04:24:44.911019520,0.0,1,10.029449,1.028661
3,886407,2020-05-06 04:24:44.911019520,892571,2020-05-06 04:24:54.940468480,0.0,1,10.029449,1.028661
4,886407,2020-05-06 04:24:54.940468480,892571,2020-05-06 04:25:04.969917440,0.0,1,10.029449,1.028661
5,886407,2020-05-06 04:25:04.969917440,892571,2020-05-06 04:25:14.999366144,0.0,1,10.029449,1.028661
6,905862,2020-05-06 04:27:24.721360896,907345,2020-05-06 04:27:39.137323008,0.0,1,14.415962,1.521026
7,908263,2020-05-06 04:27:48.098642176,914723,2020-05-06 04:27:58.594233088,0.0,1,10.495591,1.076471
8,908263,2020-05-06 04:27:58.594233088,914723,2020-05-06 04:28:09.089824000,0.0,1,10.495591,1.076471
9,908263,2020-05-06 04:28:09.089824000,914723,2020-05-06 04:28:19.585415168,0.0,1,10.495591,1.076471


In [40]:
pp1_pxstr_file = pd.read_csv('PP2_prox0506.csv',usecols=['time','PP2Entry'])
pp1_pxstr_file=pp1_pxstr_file.rename(columns={'time':'timestamp'})
pp1_pxstr_file

,timestamp,PP2Entry
0,2020-05-06T08:21:33.002660096Z,1
1,2020-05-06T08:21:33.046422016Z,1
2,2020-05-06T08:21:33.136542976Z,1
3,2020-05-06T08:52:20.048254976Z,1
4,2020-05-06T08:53:19.732531968Z,1
5,2020-05-06T08:53:19.796463872Z,1
6,2020-05-06T08:53:19.9457792Z,1
7,2020-05-06T08:54:22.061594112Z,1
8,2020-05-06T08:54:22.129753088Z,1
9,2020-05-06T08:59:20.20893184Z,1


In [43]:
###### TRYING TO ONLY INCLUDE THOSE WHICH HAVE A PROX TRIGGER  ######

# spike_df  = pp1_pxstr_file.query('PP2Entry == 1').loc[:,['timestamp']]
# spike_arr = np.array(spike_df.timestamp)

In [44]:
# i = len(pcb_merge_pp1) - 1
# END_FLAG = 0
# REMOVE_FLAG = 0

In [45]:
# while (END_FLAG == 0):
#     REMOVE_FLAG = 0                             #initialization
#     if (i < 0):
#         END_FLAG = 1
#     elif (i == 0):
#         REMOVE_FLAG = 0
#     else:
#         if (pcb_merge_pp1.weightage[i] <= THRESHOLD):
#             str_tmstmp = str(pd.Timestamp(pcb_merge_pp1.dptr_tmstmp[i]) - pd.Timedelta('0 days 00:00:10.000000'))
#             str_tmstmp = str_tmstmp.replace(' ','T')                
#             end_tmstmp = str(pd.Timestamp(pcb_merge_pp1.dptr_tmstmp[i]) + pd.Timedelta('0 days 00:00:10.000000'))              
#             end_tmstmp = end_tmstmp.replace(' ','T')
            
#             spk_i_num = np.sum((spike_arr >= str_tmstmp) & (spike_arr <= end_tmstmp))
            
#             print(i, '|', str_tmstmp, ':', end_tmstmp, '|', spk_i_num)
            
#             if spk_i_num == 0:
#                 if (pcb_merge_pp1.weightage[i-1] <= THRESHOLD):
#                     str_tmstmp = str(pd.Timestamp(pcb_merge_pp1.dptr_tmstmp[i-1]) - pd.Timedelta('0 days 00:00:05.000000'))
#                     str_tmstmp = str_tmstmp.replace(' ','T')                
#                     end_tmstmp = str(pd.Timestamp(pcb_merge_pp1.dptr_tmstmp[i-1]) + pd.Timedelta('0 days 00:00:05.000000'))              
#                     end_tmstmp = end_tmstmp.replace(' ','T')
                    
#                     spk_iml_num = np.sum((spike_arr >= str_tmstmp) & (spike_arr <= end_tmstmp))
                    
#                     print((i-1), '|', str_tmstmp, ':', end_tmstmp, '|', spk_iml_num)
                    
#                     if (spk_iml_num == 0):
#                         cumul_weightage = pcb_merge_pp1.weightage[i] + pcb_merge_pp1.weightage[i-1]
#                         if (cumul_weightage >= 0.75):
#                             REMOVE_FLAG =1
#                             pcb_merge_pp1 = pcb_merge_pp1.drop(pcb_merge_pp1.index[[i,i-1]])

In [41]:
PROC_ONE_END_FLAG = 0
while (PROC_ONE_END_FLAG == 0):
    
    iter_one = iter_one + 1
    print('PROC-ONE-ITERATION :', iter_one)
    
    pcb_merge_pp1,mrg_binry_pp1,no_of_merged_pcbs = routine_rev_merge_pxmty_algoPP1.rev_merge_pxmty_algo    \
    (inp_df        = pcb_merge_pp1                                      \
    ,pxmty_df      = pp1_pxstr_file      \
    ,inp_binry_sig = mrg_binry_pp1                         \
    ,THRESHOLD     = THRESHOLD                                               \
    ,HIGH_END      = HIGH_END                                                \
    )
    
    if (no_of_merged_pcbs == 0):
        PROC_ONE_END_FLAG = 1
    #end-if
    
    print ('=============================================')

PROC-ONE-ITERATION : 1
START :  2020-05-08 18:20:41.965805
No. of merged PCBS = 0
END   :  2020-05-08 18:20:42.812660
Execution time = 0.01411425 mins


In [42]:
pcb_merge_pp1

,arvl_index,arvl_tmstmp,dptr_index,dptr_tmstmp,no_of_parts,proc_dur,maint_dur,weightage
0,886407,2020-05-06 04:24:14.822673152,892571,2020-05-06 04:24:24.852122112,1,10.029449,0.0,1.028661
1,886407,2020-05-06 04:24:24.852122112,892571,2020-05-06 04:24:34.881570816,1,10.029449,0.0,1.028661
2,886407,2020-05-06 04:24:34.881570816,892571,2020-05-06 04:24:44.911019520,1,10.029449,0.0,1.028661
3,886407,2020-05-06 04:24:44.911019520,892571,2020-05-06 04:24:54.940468480,1,10.029449,0.0,1.028661
4,886407,2020-05-06 04:24:54.940468480,892571,2020-05-06 04:25:04.969917440,1,10.029449,0.0,1.028661
5,886407,2020-05-06 04:25:04.969917440,892571,2020-05-06 04:25:14.999366144,1,10.029449,0.0,1.028661
6,905862,2020-05-06 04:27:24.721360896,907345,2020-05-06 04:27:39.137323008,1,14.415962,0.0,1.521026
7,908263,2020-05-06 04:27:48.098642176,914723,2020-05-06 04:27:58.594233088,1,10.495591,0.0,1.076471
8,908263,2020-05-06 04:27:58.594233088,914723,2020-05-06 04:28:09.089824000,1,10.495591,0.0,1.076471
9,908263,2020-05-06 04:28:09.089824000,914723,2020-05-06 04:28:19.585415168,1,10.495591,0.0,1.076471


In [43]:
PROC_TWO_END_FLAG = 0
while(PROC_TWO_END_FLAG == 0):
    
    iter_two = iter_two + 1
    print('PROC-TWO-ITERATION :', iter_two)
    
    pcb_merge_pp1,mrg_binry_pp1,no_of_merged_pcbs = routine_merging_algoPP1.merging_algo  \
    (inp_df        = pcb_merge_pp1                                   \
    ,inp_binry_sig = mrg_binry_pp1                                   \
    ,THRESHOLD     = THRESHOLD                                            \
    ,HIGH_END      = HIGH_END                                             \
    ) 
    
    if(no_of_merged_pcbs == 0):
        PROC_TWO_END_FLAG = 1
    #end-if
    
    print ('=============================================')

PROC-TWO-ITERATION : 1
START :  2020-05-08 18:20:43.252481
No. of merged PCBS = 0
END   :  2020-05-08 18:20:43.979881
Execution time = 0.012123333333333335 mins


In [44]:
pcb_merge_pp1

,arvl_index,arvl_tmstmp,dptr_index,dptr_tmstmp,no_of_parts,proc_dur,maint_dur,weightage
0,886407.0,2020-05-06 04:24:14.822673152,892571.0,2020-05-06 04:24:24.852122112,1.0,10.029449,0.0,1.028661
1,886407.0,2020-05-06 04:24:24.852122112,892571.0,2020-05-06 04:24:34.881570816,1.0,10.029449,0.0,1.028661
2,886407.0,2020-05-06 04:24:34.881570816,892571.0,2020-05-06 04:24:44.911019520,1.0,10.029449,0.0,1.028661
3,886407.0,2020-05-06 04:24:44.911019520,892571.0,2020-05-06 04:24:54.940468480,1.0,10.029449,0.0,1.028661
4,886407.0,2020-05-06 04:24:54.940468480,892571.0,2020-05-06 04:25:04.969917440,1.0,10.029449,0.0,1.028661
5,886407.0,2020-05-06 04:25:04.969917440,892571.0,2020-05-06 04:25:14.999366144,1.0,10.029449,0.0,1.028661
6,905862.0,2020-05-06 04:27:24.721360896,907345.0,2020-05-06 04:27:39.137323008,1.0,14.415962,0.0,1.521026
7,908263.0,2020-05-06 04:27:48.098642176,914723.0,2020-05-06 04:27:58.594233088,1.0,10.495591,0.0,1.076471
8,908263.0,2020-05-06 04:27:58.594233088,914723.0,2020-05-06 04:28:09.089824000,1.0,10.495591,0.0,1.076471
9,908263.0,2020-05-06 04:28:09.089824000,914723.0,2020-05-06 04:28:19.585415168,1.0,10.495591,0.0,1.076471


In [50]:
filt_sig_pp1['mrg_binry_sig'] = mrg_binry_pp1

In [51]:
pp1EMfile = pd.read_csv('PP1_EM0506.csv', usecols=['time', 'c1', 'c2', 'c3'])

In [52]:
raw_pp_current=pp1EMfile

In [53]:
raw_pp_current['time']=pd.to_datetime(raw_pp_current['time'])
raw_pp_current=raw_pp_current.rename(columns={'time':'timestamp'})

In [54]:
raw_pp_current

,timestamp,c1,c2,c3
0,2020-03-11 02:00:00.752566016,0.0,0.0,0.0
1,2020-03-11 02:00:02.607884032,0.0,0.0,0.0
2,2020-03-11 02:00:04.447577088,0.0,0.0,0.0
3,2020-03-11 02:00:06.303332096,0.0,0.0,0.0
4,2020-03-11 02:00:08.111015936,0.0,0.0,0.0
5,2020-03-11 02:00:09.950635008,0.0,0.0,0.0
6,2020-03-11 02:00:11.806308096,0.0,0.0,0.0
7,2020-03-11 02:00:13.646241024,0.0,0.0,0.0
8,2020-03-11 02:00:15.502080000,0.0,0.0,0.0
9,2020-03-11 02:00:17.341309184,0.0,0.0,0.0


In [55]:
raw_pp_current['total_current']=raw_pp_current['c1']+raw_pp_current['c2']+raw_pp_current['c3']

In [56]:
working_times_df=pd.DataFrame()

working_times_df['timestamp']=pcb_merge_pp1['arvl_tmstmp']
working_times_df['working_time']=pcb_merge_pp1['proc_dur']
working_times_df['event']=1

In [57]:
# working_times_df.reset_index(inplace=True)
# working_times_df.drop('index', axis=1, inplace=True)
# working_times_df.drop('level_0', axis=1, inplace=True)
# working_times_df

In [58]:
working_times_df['energy'] = pcb_merge_pp1.apply(lambda x: raw_pp_current.loc[(raw_pp_current.timestamp <= x.dptr_tmstmp) & (x.arvl_tmstmp <= raw_pp_current.timestamp), ['total_current']].sum()*230/3600000, axis=1)

In [59]:
working_times_df

,timestamp,working_time,event,energy
0,2020-03-11 03:49:57.561816064,55.087314,1,0.086123
1,2020-03-11 04:23:04.511490048,70.256957,1,0.006236
2,2020-03-11 04:28:55.276083968,60.300515,1,0.005675
3,2020-03-11 04:33:25.495811072,66.690853,1,0.005883
4,2020-03-11 04:36:31.088248064,73.416887,1,0.047979
5,2020-03-11 04:51:16.583371008,70.148652,1,0.006584
6,2020-03-11 04:52:38.727220992,70.154401,1,0.006439
7,2020-03-11 04:53:52.099322112,70.927654,1,0.006350
8,2020-03-11 04:55:03.420464896,71.127943,1,0.006027
9,2020-03-11 04:56:16.156135168,75.367641,1,0.007343


In [60]:
PP_temp_idle=filt_sig_pp1[['timestamp', 'mrg_binry_sig']]
PP_temp_idle['mrg_binry_sig']=PP_temp_idle['mrg_binry_sig']*-1

/home/raghu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [61]:
PP_temp_idle

,timestamp,mrg_binry_sig
0,2020-03-11 02:00:01.498893056,-0.0
1,2020-03-11 02:00:01.499866880,-0.0
2,2020-03-11 02:00:01.500781056,-0.0
3,2020-03-11 02:00:01.501683968,-0.0
4,2020-03-11 02:00:01.502603008,-0.0
5,2020-03-11 02:00:01.545491968,-0.0
6,2020-03-11 02:00:01.546955008,-0.0
7,2020-03-11 02:00:01.548336896,-0.0
8,2020-03-11 02:00:01.549330944,-0.0
9,2020-03-11 02:00:01.550308864,-0.0


In [62]:
PP_temp_idle['mrg_binry_sig'].iloc[0]=(PP_temp_idle['mrg_binry_sig'].iloc[1]+1)%2*-1
PP_temp_idle['mrg_binry_sig'].iloc[-1]=(PP_temp_idle['mrg_binry_sig'].iloc[-2]+1)%2*-1

/home/raghu/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
/home/raghu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/raghu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [63]:
PP_temp_idle

,timestamp,mrg_binry_sig
0,2020-03-11 02:00:01.498893056,-1.0
1,2020-03-11 02:00:01.499866880,-0.0
2,2020-03-11 02:00:01.500781056,-0.0
3,2020-03-11 02:00:01.501683968,-0.0
4,2020-03-11 02:00:01.502603008,-0.0
5,2020-03-11 02:00:01.545491968,-0.0
6,2020-03-11 02:00:01.546955008,-0.0
7,2020-03-11 02:00:01.548336896,-0.0
8,2020-03-11 02:00:01.549330944,-0.0
9,2020-03-11 02:00:01.550308864,-0.0


In [64]:
idle_times_raw=sp.signal.find_peaks(PP_temp_idle.mrg_binry_sig, height=(-0.4), width=1)
idle_times_raw_df=pd.DataFrame({"sample_number":idle_times_raw[0], "working_time":idle_times_raw[1]['widths']/100})

In [65]:
idle_times_raw_df

,sample_number,working_time
0,222727,4454.54
1,554634,1728.42
2,655171,20.48
3,671451,29.71
4,676265,0.53
5,862525,0.74
6,868865,1.73
7,879608,7.27
8,901767,274.47
9,932306,214.62


In [66]:
for x, row in idle_times_raw_df.iterrows():
    idle_times_raw_df.ix[x,'timestamp']=PP_temp_idle.iloc[int(idle_times_raw[0][x])].timestamp
    idle_times_raw_df.ix[x,'start_time']=PP_temp_idle.iloc[int(idle_times_raw[1]['left_ips'][x])].timestamp
    idle_times_raw_df.ix[x,'end_time']=PP_temp_idle.iloc[int(idle_times_raw[1]['right_ips'][x])].timestamp
    

/home/raghu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  
/home/raghu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  This is separate from the ipykernel package so we can avoid doing imports until
/home/raghu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  after remo

In [67]:
idle_times_raw_df['event']=0

In [68]:
idle_times_raw_df['energy'] = idle_times_raw_df.apply(lambda x: raw_pp_current.loc[(raw_pp_current.timestamp <= x.end_time) & 
                                                            (x.start_time <= raw_pp_current.timestamp),
                                                            ['total_current']].sum()*230/3600000, axis=1)

In [69]:
idle_times_raw_df

,sample_number,working_time,timestamp,start_time,end_time,event,energy
0,222727,4454.54,2020-03-11 02:36:13.742492928,2020-03-11 02:00:01.498893056,2020-03-11 03:12:26.032560128,0,0.022416
1,554634,1728.42,2020-03-11 03:30:10.796452096,2020-03-11 03:16:08.012131072,2020-03-11 03:44:13.873709056,0,0.092355
2,655171,20.48,2020-03-11 03:46:31.594790912,2020-03-11 03:46:21.602196992,2020-03-11 03:46:41.587749888,0,0.001114
3,671451,29.71,2020-03-11 03:49:10.371102976,2020-03-11 03:48:55.890827008,2020-03-11 03:49:24.849998080,0,0.001756
4,676265,0.53,2020-03-11 03:49:57.315438080,2020-03-11 03:49:57.026298112,2020-03-11 03:49:57.560849920,0,0.000000
5,862525,0.74,2020-03-11 04:20:14.225755904,2020-03-11 04:20:13.882669056,2020-03-11 04:20:14.612413952,0,0.000100
6,868865,1.73,2020-03-11 04:21:16.092164096,2020-03-11 04:21:15.261511168,2020-03-11 04:21:16.921510912,0,0.000098
7,879608,7.27,2020-03-11 04:23:00.954164992,2020-03-11 04:22:57.396261888,2020-03-11 04:23:04.467017984,0,0.000410
8,901767,274.47,2020-03-11 04:26:38.922700800,2020-03-11 04:24:23.197207808,2020-03-11 04:28:55.275163904,0,0.015794
9,932306,214.62,2020-03-11 04:31:40.832498944,2020-03-11 04:29:55.532219904,2020-03-11 04:33:25.494225920,0,0.010277


In [70]:
idle_times_raw_df['energy'].sum(axis = 0, skipna = True) 

1.0054565

In [41]:
# idle_times_raw_df.drop('sample_number', axis=1, inplace=True)
# idle_times_raw_df.drop('start_time', axis=1, inplace=True)
# idle_times_raw_df.drop('end_time', axis=1, inplace=True)

In [42]:
# idle_times_raw_df.reset_index(inplace=True)
# idle_times_raw_df.drop('index', axis=1, inplace=True)
# idle_times_raw_df.drop('level_0', axis=1, inplace=True)
# idle_times_raw_df

In [71]:
working_times_df=working_times_df[['event', 'working_time', 'energy','timestamp']]
idle_times_df=idle_times_raw_df[['event', 'working_time', 'energy', 'timestamp']]

In [72]:
working_times_df

,event,working_time,energy,timestamp
0,1,55.087314,0.086123,2020-03-11 03:49:57.561816064
1,1,70.256957,0.006236,2020-03-11 04:23:04.511490048
2,1,60.300515,0.005675,2020-03-11 04:28:55.276083968
3,1,66.690853,0.005883,2020-03-11 04:33:25.495811072
4,1,73.416887,0.047979,2020-03-11 04:36:31.088248064
5,1,70.148652,0.006584,2020-03-11 04:51:16.583371008
6,1,70.154401,0.006439,2020-03-11 04:52:38.727220992
7,1,70.927654,0.006350,2020-03-11 04:53:52.099322112
8,1,71.127943,0.006027,2020-03-11 04:55:03.420464896
9,1,75.367641,0.007343,2020-03-11 04:56:16.156135168


In [73]:
idle_times_df

,event,working_time,energy,timestamp
0,0,4454.54,0.022416,2020-03-11 02:36:13.742492928
1,0,1728.42,0.092355,2020-03-11 03:30:10.796452096
2,0,20.48,0.001114,2020-03-11 03:46:31.594790912
3,0,29.71,0.001756,2020-03-11 03:49:10.371102976
4,0,0.53,0.000000,2020-03-11 03:49:57.315438080
5,0,0.74,0.000100,2020-03-11 04:20:14.225755904
6,0,1.73,0.000098,2020-03-11 04:21:16.092164096
7,0,7.27,0.000410,2020-03-11 04:23:00.954164992
8,0,274.47,0.015794,2020-03-11 04:26:38.922700800
9,0,214.62,0.010277,2020-03-11 04:31:40.832498944


In [74]:
idle_times_df['energy'].sum(axis = 0, skipna = True) 

1.0054565

In [75]:
working_times_df['energy'].sum(axis = 0, skipna = True)

0.5339807777777777

In [48]:
events_batch=[]

# events_batch = json.loads(LL_events.to_json())

for i,row in idle_times_df.iterrows():
    events_batch.append({ 'measurement': 'EVENTS','tags':{'Machine':'PP1'}, 'time':row.timestamp ,'fields': {'PP1 Idle Event':int(row.event),'PP1 Idle Time':float(row.working_time),"PP1 Idle Energy":float(row.energy)}})
    
for i,row in working_times_df.iterrows():
    events_batch.append({ 'measurement': 'EVENTS','tags':{'Machine':'PP1'}, 'time':row.timestamp ,'fields': {'PP1 Processing Event':int(row.event),'PP1 Processing Time':float(row.working_time),"PP1 Processing Energy":float(row.energy)}})

In [49]:
# PP_events=pd.DataFrame()

In [50]:
# PP_events=PP_events.append(working_times_df)

In [51]:
# PP_events=PP_events.append(idle_times_df)

In [52]:
# PP_events

,event,working_time,energy,timestamp
0,1,75.560252,0.118920,2020-03-11 03:12:26.033467904
1,1,58.206421,0.012363,2020-03-11 03:46:41.588787200
2,1,86.293456,0.090517,2020-03-11 03:49:57.561816064
3,1,19.844322,0.005560,2020-03-11 04:21:16.965949952
4,1,70.256957,0.006236,2020-03-11 04:23:04.511490048
5,1,60.300515,0.005675,2020-03-11 04:28:55.276083968
6,1,78.439544,0.006514,2020-03-11 04:33:25.495811072
7,1,73.416887,0.047979,2020-03-11 04:36:31.088248064
8,1,70.148652,0.006584,2020-03-11 04:51:16.583371008
9,1,70.154401,0.006439,2020-03-11 04:52:38.727220992
